In [5]:
from LoadVars import *
from ga import *
from NeuralNet import NeuralNet, train, calculate
from performance import calc_hv
from SaveOutput import save
import matplotlib.pyplot as plt
import torch

In [6]:
#Defining problem
problem = define_problem(problem_name)
pareto_front = problem.pareto_front()
save('OUTPUT/pareto_front.dat', problem.pareto_front(), header=f'Pareto Front of {problem_name}')
print(f'The benchmark problem: {problem_name.upper()}\n')

The benchmark problem: OSY



In [7]:
#Initial sampling
initial_sampling = define_sampling(initial_sampling_method_name)
print(f'Performing initial sampling: {initial_sampling_method_name.upper()}\n')
InitialData = initial_sampling.do(problem, n_samples=pop_size, pop=None)

Performing initial sampling: REAL_LHS



In [8]:
#Evaluating initial samples (true eval)
InitialEval_F = problem.evaluate(InitialData, return_values_of=['F'])
InitialEval_G = problem.evaluate(InitialData, return_values_of=['G'])
InitialEval_CV = problem.evaluate(InitialData, return_values_of=['CV'])

if InitialEval_G is not None:
	InitialEval = np.concatenate((InitialEval_F, InitialEval_G, InitialEval_CV), axis=1)

parent_pop = set_individual(X=InitialData,
							F=InitialEval_F,
							G=InitialEval_G,
							CV=InitialEval_CV)

parent_pop = set_population_from_array_or_individual(parent_pop)

In [9]:
HV = [0.0]
HV += [calc_hv(InitialEval[:,range(problem.n_obj)], ref=hv_ref)]

print(HV)

[0.0, 0.0]


In [10]:
#Initial training for neural nets
print('Feeding the training data to the neural net...\n\n')

Model = NeuralNet(D_in=problem.n_var,
				  H=N_Neuron, D=N_Neuron,
				  D_out=problem.n_obj+problem.n_constr)

print('Performing initial training...\n')

TrainedModel = train(problem=problem,
					 model=Model,
				     N_Epoch=N_Epoch,
				     lr=lr,
				     batchrate=batchrate)

print('\nAn initial trained model is obtained!\n')
print('--------------------------------------------------')
TrainedModel_Problem = TrainedModelProblem(problem, TrainedModel)

Feeding the training data to the neural net...


Performing initial training...

Processing the data... please wait :)

N_Epoch = 0, Train Loss = 64.45493379441818, Valid Loss = 111.32809448242188
N_Epoch = 50, Train Loss = 0.17703580697721774, Valid Loss = 1.4480007886886597

An initial trained model is obtained!

--------------------------------------------------


In [11]:
#Evolutionary computation routines on the Trained Model
selection = define_selection(selection_operator_name)
crossover = define_crossover(crossover_operator_name, prob=prob_c, eta=eta_c)
mutation = define_mutation(mutation_operator_name, eta=eta_m)

#EA settings
EA = EvolutionaryAlgorithm(algorithm_name)
algorithm = EA.setup(pop_size=pop_size,
					 sampling=initial_sampling,
					 # selection=selection,
					 crossover=crossover,
					 mutation=mutation)

#Stopping criteria
stopping_criteria_def = StoppingCriteria(termination_name)
stopping_criteria = stopping_criteria_def.set_termination(n_gen=n_gen)

#Obtaining optimal solutions on the initial trained model
print(f'Performing optimization on the initial trained model using {algorithm_name.upper()}\n')
optimal_solutions =  do_optimization(TrainedModel_Problem,
									 algorithm, stopping_criteria,
									 verbose=True, seed=1,
									 return_least_infeasible=False)
print('--------------------------------------------------')
print('\nOptimal solutions on the initial trained model is obtained!\n')
print('--------------------------------------------------')

Performing optimization on the initial trained model using NSGA2

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     100 |  0.00000E+00 |  1.562225844 |       2 |            - |            -
    2 |     200 |  0.00000E+00 |  0.534462572 |       3 |  0.070293365 |            f
    3 |     300 |  0.00000E+00 |  0.104923719 |       4 |    0.3925286 |        ideal
    4 |     400 |  0.00000E+00 |  0.002211254 |      11 |   0.19629687 |        ideal
    5 |     500 |  0.00000E+00 |  0.00000E+00 |       9 |  0.029585099 |        ideal
    6 |     600 |  0.00000E+00 |  0.00000E+00 |       6 |     0.200428 |        ideal
    7 |     700 |  0.00000E+00 |  0.00000E+00 |       7 |    0.2585609 |        ideal
    8 |     800 |  0.00000E+00 |  0.00000E+00 |      13 | 0.0087128375 |        ideal
    9 |     900 |  0.00000E+00 |  0.00000E+00 |      15 |   0.13140918 |        ideal
   10 |    1000 |  0.00000E+00 |  0.00000E+00 |      18 |  0.082060196 |  

In [15]:
child_pop = set_individual(X=optimal_solutions.X,
						   F=optimal_solutions.F,
						   G=optimal_solutions.G,
						   CV=optimal_solutions.CV)

child_pop = set_population_from_array_or_individual(child_pop)

survivors = Population.merge(parent_pop, child_pop)

print(survivors.get('F').shape)

(2, 100, 2)
